<a href="https://colab.research.google.com/github/mattiadido95/Haze-Fog-suppression/blob/main/dataset_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
from google.colab import drive
from google.colab.patches import cv2_imshow
import os
import gdown
import zipfile
import cv2
import numpy as np

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
folder_path = "drive/MyDrive/Haze-Fog-suppression"
os.chdir(folder_path)

# Uncompress Zip files


In [ ]:
os.chdir("dataset")
current_directory = os.getcwd()

#estrae tutti i file zip e cancella il file compresso
for file_name in os.listdir(current_directory):
    if file_name.endswith(".zip"):
        path_to_zip_file = os.path.join(current_directory, file_name)
        with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
            zip_ref.extractall(current_directory)
        os.remove(path_to_zip_file)

# Generate datasets

In [ ]:
#`d` sia l'immagine di profondità e `thresholds` sia una lista di tre soglie
def compute_transmission_map_Koschmieder(d):
    H = d.shape[0]
    W = d.shape[1]
    vector = d.reshape(H * W * 3)
    for pixel_value in vector:
      if pixel_value < 50:
        pixel_value = 0
      elif pixel_value > 50 and pixel_value < 87:
        pixel_value = 50
      elif pixel_value > 87 and pixel_value < 167:
        pixel_value = 87
      elif pixel_value > 167 and pixel_value < 209:
        pixel_value = 167
      else:
        pixel_value = 255
    normalized = vector.reshape(H, W, 3).astype(np.uint8)
    t = np.exp(-0.01 * normalized)
    return t

def compute_transmission_map_Bruneton(d):
  beta = 0.008
  cos_theta = -0.8
  epsilon = 2
  t = np.exp(-beta * d / pow(cos_theta, epsilon))
  return t


In [ ]:
def compute_cevr(original_image,degraded_image):
    max_J = np.max(original_image)
    min_J = np.min(original_image)
    max_I = np.max(degraded_image)
    min_I = np.min(degraded_image)

    cevr = (max_J - min_J) / (max_I - min_I)
    return cevr

In [4]:
# Specifica i parametri
# Valore di luce atmosferica (generalmente tra 0 e 255)
low_A = 120
medium_A = 180
high_A = 220

low_beta = 0.004
medium_beta = 0.006
high_beta = 0.008
very_high_beta = 0.01

path_to_original_images = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/Vis/"
path_to_depth_maps = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/DepthMaps/"
path_to_input = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/train_A/"
path_to_output = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/train_B/"

In [ ]:
def generate_fog_image(path_to_original_image,path_to_depth_map,A,beta,img_name,level):
    # Carica l'immagine originale e la depth map
    J = cv2.imread(path_to_original_image)
    d = cv2.imread(path_to_depth_map)

    t = np.exp(-beta * d)
    I = J * t + A * (1 - t)

    cv2.imwrite(path_to_input + img_name[:-4] + "_" + str(level) + '.png', np.uint8(I))
    cv2.imwrite(path_to_output + img_name[:-4] + "_" + str(level) + '.png', np.uint8(J))

In [ ]:
# Scorri tutti i file nella cartella
file_ordinati = sorted(os.listdir(path_to_original_images))

for nome_file in file_ordinati[:2100]:
    path_to_original_image = os.path.join(path_to_original_images, nome_file)
    path_to_depth_map = os.path.join(path_to_depth_maps, nome_file)
    generate_fog_image(path_to_original_image,path_to_depth_map,low_A,low_beta,nome_file,"low")
    generate_fog_image(path_to_original_image,path_to_depth_map,medium_A,medium_beta,nome_file,"medium")
    generate_fog_image(path_to_original_image,path_to_depth_map,high_A,high_beta,nome_file,"high")

In [ ]:
# Scorri tutti i file nella cartella
file_ordinati = sorted(os.listdir(path_to_original_images))

for nome_file in file_ordinati[2100:]:
    path_to_original_image = os.path.join(path_to_original_images, nome_file)
    path_to_depth_map = os.path.join(path_to_depth_maps, nome_file)
    generate_fog_image(path_to_original_image,path_to_depth_map,low_A,low_beta,nome_file,"low")
    generate_fog_image(path_to_original_image,path_to_depth_map,medium_A,medium_beta,nome_file,"medium")
    generate_fog_image(path_to_original_image,path_to_depth_map,high_A,high_beta,nome_file,"high")

# Copy file into Pix2PixHD model

### seleziona randomicamente 4200 immagini con livello di nebbia differente

In [ ]:
lista_file = [file for file in os.listdir(path_to_original_images)]
classi = ["low", "medium", "high"]
train = []
test = []

for i in range(len(lista_file)):
    livello = i % 3
    livello_comp1 = (i+1) % 3
    livello_comp2 = (i+2) % 3
    train.append(str(lista_file[i][:-4])+"_"+str(classi[livello])+".png")
    test.append(str(lista_file[i][:-4])+"_"+str(classi[livello_comp1]+".png"))
    test.append(str(lista_file[i][:-4])+"_"+str(classi[livello_comp2]+".png"))

print(train[:10])
print(test[:10])
print(len(train))
print(len(test))


['03200_low.png', '03201_medium.png', '03202_high.png', '03203_low.png', '03204_medium.png', '03205_high.png', '03206_low.png', '03207_medium.png', '03208_high.png', '03209_low.png']
['03200_medium.png', '03200_high.png', '03201_high.png', '03201_low.png', '03202_low.png', '03202_medium.png', '03203_medium.png', '03203_high.png', '03204_high.png', '03204_low.png']
4200
8400


In [ ]:
import shutil

# Definisci i percorsi delle cartelle sorgente e destinazione
percorso_sorgente = '/content/drive/MyDrive/Haze-Fog-suppression/dataset/train_A/'
percorso_destinazione = '/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_A'

# Crea la cartella destinazione se non esiste
if not os.path.exists(percorso_destinazione):
    os.makedirs(percorso_destinazione)

# Copia i file dalla cartella sorgente alla destinazione
for file in train:
    percorso_file_sorgente = os.path.join(percorso_sorgente, file)
    percorso_file_destinazione = os.path.join(percorso_destinazione, file)
    shutil.copy2(percorso_file_sorgente, percorso_file_destinazione)


In [ ]:
import shutil

# Definisci i percorsi delle cartelle sorgente e destinazione
percorso_sorgente = '/content/drive/MyDrive/Haze-Fog-suppression/dataset/train_B/'
percorso_destinazione = '/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_B'

# Crea la cartella destinazione se non esiste
if not os.path.exists(percorso_destinazione):
    os.makedirs(percorso_destinazione)

# Copia i file dalla cartella sorgente alla destinazione
for file in train:
    percorso_file_sorgente = os.path.join(percorso_sorgente, file)
    percorso_file_destinazione = os.path.join(percorso_destinazione, file)
    shutil.copy2(percorso_file_sorgente, percorso_file_destinazione)


'/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_B'

In [ ]:
import os

# Definisci il percorso della cartella di destinazione
percorso_destinazione_A = '/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_A'
percorso_destinazione_B = '/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_B'

# Stampa il numero di file
print("Il numero totale di file nella cartella train_A è:", len(os.listdir(percorso_destinazione_A)))
print("Il numero totale di file nella cartella train_B è:", len(os.listdir(percorso_destinazione_B)))

Il numero totale di file nella cartella train_A è: 12600
Il numero totale di file nella cartella train_B è: 12600


## delete dataset

In [ ]:
# DANGER
# delete train_A or train_B
import shutil
percorso_cartella_da_eliminare = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_A"
shutil.rmtree(percorso_cartella_da_eliminare)

In [ ]:
shutil.rmtree('/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_A/.ipynb_checkpoints')
shutil.rmtree('/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_B/.ipynb_checkpoints')

In [18]:
file_ordinati = sorted(os.listdir(percorso_destinazione_A))
print(file_ordinati)

['00000_medium.png', '00001_high.png', '00002_low.png', '00003_medium.png', '00004_high.png', '00005_low.png', '00006_medium.png', '00007_high.png', '00008_low.png', '00009_medium.png', '00010_high.png', '00011_low.png', '00012_medium.png', '00013_high.png', '00014_low.png', '00015_medium.png', '00016_high.png', '00017_low.png', '00018_medium.png', '00019_high.png', '00020_low.png', '00021_medium.png', '00022_high.png', '00023_low.png', '00024_medium.png', '00025_high.png', '00026_low.png', '00027_medium.png', '00028_high.png', '00029_low.png', '00030_medium.png', '00031_high.png', '00032_low.png', '00033_medium.png', '00034_high.png', '00035_low.png', '00036_medium.png', '00037_high.png', '00038_low.png', '00039_medium.png', '00040_high.png', '00041_low.png', '00042_medium.png', '00043_high.png', '00044_low.png', '00045_medium.png', '00046_high.png', '00047_low.png', '00048_medium.png', '00049_high.png', '00050_low.png', '00051_medium.png', '00052_high.png', '00053_low.png', '00054_me

# Resize images


In [ ]:
#Check dimension image ---> fatto in locale perche impiegava troppo tempo

# Definisci il percorso della cartella di destinazione
percorso_destinazione_A = '/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_A/'
percorso_destinazione_B = '/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_B/'

file_ordinati = sorted(os.listdir(percorso_destinazione_A))
count=0
file = open("lista_img.txt", "w")
for nome_file in file_ordinati:
    input = os.path.join(percorso_destinazione_A, nome_file)
    output = os.path.join(percorso_destinazione_B, nome_file)
    i = cv2.imread(input)
    o = cv2.imread(output)
    if i.shape != (768, 1024, 3):
      file.write(nome_file + "\n")
    if o.shape != (768, 1024, 3):
      file.write(nome_file + "\n")
    count += 1
    if count % 1000 == 0:
      print(count)
file.close()


KeyboardInterrupt: ignored

In [5]:
#Resize images already processed

# Definisci il percorso della cartella di destinazione
percorso_destinazione_A = '/content/drive/MyDrive/Haze-Fog-suppression/dataset/train_A/'
percorso_destinazione_B = '/content/drive/MyDrive/Haze-Fog-suppression/dataset/train_B/'
new_width = 1024
new_height = 768


#lettura file da modificare
file = open("lista_img.txt", "r")
lines = file.readlines()
lines = [line.rstrip() for line in lines]
file.close()

for nome_file in lines:
  input_low = percorso_destinazione_A + nome_file[:-4] + "_" + 'low.png'
  output_low = percorso_destinazione_B + nome_file[:-4] + "_" + 'low.png'
  input_medium = percorso_destinazione_A + nome_file[:-4] + "_" + 'medium.png'
  output_medium = percorso_destinazione_B + nome_file[:-4] + "_" + 'medium.png'
  input_high = percorso_destinazione_A + nome_file[:-4] + "_" + 'high.png'
  output_high = percorso_destinazione_B + nome_file[:-4] + "_" + 'high.png'

  # Per l'immagine di input di bassa qualità
  image_input_low = cv2.imread(input_low)
  resized_image_input_low = cv2.resize(image_input_low, (new_width, new_height))
  os.remove(input_low)
  cv2.imwrite(input_low, resized_image_input_low)

  # Per l'immagine di output di bassa qualità
  image_output_low = cv2.imread(output_low)
  resized_image_output_low = cv2.resize(image_output_low, (new_width, new_height))
  os.remove(output_low)
  cv2.imwrite(output_low, resized_image_output_low)

  # Per l'immagine di input di media qualità
  image_input_medium = cv2.imread(input_medium)
  resized_image_input_medium = cv2.resize(image_input_medium, (new_width, new_height))
  os.remove(input_medium)
  cv2.imwrite(input_medium, resized_image_input_medium)

  # Per l'immagine di output di media qualità
  image_output_medium = cv2.imread(output_medium)
  resized_image_output_medium = cv2.resize(image_output_medium, (new_width, new_height))
  os.remove(output_medium)
  cv2.imwrite(output_medium, resized_image_output_medium)

  # Per l'immagine di input di alta qualità
  image_input_high = cv2.imread(input_high)
  resized_image_input_high = cv2.resize(image_input_high, (new_width, new_height))
  os.remove(input_high)
  cv2.imwrite(input_high, resized_image_input_high)

  # Per l'immagine di output di alta qualità
  image_output_high = cv2.imread(output_high)
  resized_image_output_high = cv2.resize(image_output_high, (new_width, new_height))
  os.remove(output_high)
  cv2.imwrite(output_high, resized_image_output_high)


In [ ]:
percorso_destinazione_A = '/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/nebbia/train_A/'
input_low = percorso_destinazione_B + "01301" + "_" + 'high.png'
image_input_low = cv2.imread(input_low)
print(image_input_low.shape)


(768, 1024, 3)
